# Note:
-(make sure both data1 and data2 have the same number of numerical and categorical columns)
- Run all the cells for data1
- Then, run all the cells for data2
- Then after you have the correlation matrices for both (data1 and data2) --> run the difference between matrices test
- This is the only way this file will work. If you dont follow the steps you will get a result of ZERO

In [ ]:
import numpy as np
import pandas as pd
from numpy import linalg
from sklearn.model_selection import train_test_split

# Upload Dataset

In [ ]:
# Dataset before augmentation
data1 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Stacked/stacked_orig.csv", header=None)
#data1 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Uniform/uniform_orig.csv", header=None)

# Dataset after augmentation
#data2 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Stacked/stacked_new.csv", header=None)
#data2 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Uniform/uniform_new.csv", header=None)
#data2 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Gaussian/gaussian_orig.csv", header=None)
data2 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Gaussian/gaussian_new.csv", header=None)



# Check Shapes of Datasets (they must be the same)

In [ ]:
# Print shapes for debugging
print(f"Shape of data1: {data1.shape}")
print(f"Shape of data2: {data2.shape}")


In [ ]:
# If the datasets do not have the same columns ---> make one dataset match the other one (order does not matter)
# Subset columns so both have the same 
# (we need to make sure we have the same categorical and numerical columns) ---> you do this after this cell
data1 = data1.iloc[:, -13:]
data1.shape

# Convert columns into Categorical (Only for Uniform and Gaussian Distributions)

In [ ]:
for i in range(-5,0):
    data1.iloc[:,i] = data1.iloc[:,i].astype(int) # Integer
    data1.iloc[:,i] = data1.iloc[:,i].astype('category') # Categories
data1.dtypes 

In [ ]:
for i in range(-5,0):
    data2.iloc[:,i] = data2.iloc[:,i].astype(int) # Integer
    data2.iloc[:,i] = data2.iloc[:,i].astype('category') # Categories
data2.dtypes
#data2.head()  

# Convert columns into Categorical (Only for Stacked Distribution)

In [ ]:
for column in data1.columns[-5:]:
    data1[column] = data1[column].astype('category')

# Verify the changes
print(data1.dtypes)


# Concat Synthetic Dataset with New generated dataset

In [ ]:
# Concat (adding rows)
combined_df = np.vstack((data1, data2))

# change numpy array into a dataframe
combined_data = pd.DataFrame(combined_df, columns=data1.columns)

# Convert the last number of columns to categorical (to make sure they have the same)
#for column in combined_data.columns[-5:]:
    #combined_data[column] = combined_data[column].astype('category')

# Verify the changes
print(combined_data.dtypes)

# Split Dataset for combined_data

In [ ]:
# Split dataset into X and y (data1)
X = combined_data.iloc[:, :-1]  # All columns except the last one
y = combined_data.iloc[:, -1]   # Only the last column


# Split dataset into X_train and y_train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Splitting X_train into numerical subset 
numerical_df = X_train.select_dtypes(include = ['float', 'float64'])

# Splitting X_train into categorical subset 
categorical_df = X_train.select_dtypes(exclude=['float', 'float64'])

# Split Dataset for Synthetic Data

In [ ]:
# data2
X = data1.iloc[:, :-1]
y = data1.iloc[:, -1]


# Split dataset into X_train and y_train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Splitting X_train into numerical subset 
numerical_df = X_train.select_dtypes(include = ['float', 'float64'])

# Splitting X_train into categorical subset 
categorical_df = X_train.select_dtypes(exclude=['float', 'float64'])

# Correlation between columns test (Matrix)

In [ ]:
##################### Correlation between columns (numerical) Code ############################
# Takes the X_train data to find correlation between NUMERICAL features
def num_corr(X_train_numerical):
    matrix = X_train_numerical.corr(method='pearson')
    print("---------------------------Correlation Matrix------------------------- \n", matrix)
     
#Calls the function so the matrix prints out    
correlation_matrix = num_corr(numerical_df)

# Print the dataframe to verify
#print(correlation_matrix)
#0.860


# Save Correlation Matrix into a Dataframe for combined_data

In [ ]:
#combined_data
correlation_matrix = numerical_df.corr()
correlation_df_combined = pd.DataFrame(correlation_matrix)
print(correlation_df_combined)

correlation_df_combined.shape

# Save Correlation Matrix into a Dataframe for Synthetic dataset

In [ ]:
#data1
correlation_matrix = numerical_df.corr()
correlation_df1 = pd.DataFrame(correlation_matrix)
print(correlation_df1)

correlation_df1.shape

# Differences in Matrices (Frobenius Norm)

In [ ]:
# Convert the dataframes to numpy arrays
matrix1 = correlation_df1.to_numpy()
matrix2 = correlation_df_combined.to_numpy()

# Compute the Frobenius norm of the difference between the matrices. 
frobenius_abs = np.linalg.norm(matrix1 - matrix2, ord='fro')   # Absolute error with Frobenius norm

frobenius_rel = frobenius_abs/np.linalg.norm(matrix1, ord='fro')    # Relative error with Frobenius norm

print(f"Frobenius norm difference: {frobenius_abs: .3f}")
print(f"Frobenius absolute error : {frobenius_rel .3f}")


# Differences in Arrays (L2 Norm)

In [ ]:
#array1 ...
#array2 ...

# Compute the L2 norm (Euclidean norm) of the difference between the arrays
L2_norm = np.linalg.norm(array1 - array2, ord=2)

print(f"L2 norm difference: {L2_norm: .3f}")
